In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle

In [8]:
data = pd.read_csv('SA_model_data')
data = data[['label','text']]
data.text=data.text.astype(str)

#data, final_data = train_test_split(data, test_size = 0.6, random_state = 42, stratify=data['label'])
data


,label,text
17655,3,quot never alon love realli know quot
2335,0,wait gt gg load annoy
9286,2,feel import enough live worthi enough struggl ...
13326,5,god want sleep
24733,6,drive gt 90mph daili basi cld film fast amp fu...
...,...,...
18918,3,joe photograph famili rehears alway fun first ...
18132,3,coffe gym start day right
21216,4,watch short stack tv
11048,2,inde love team incred attent ball chees live g...


In [9]:
max_fatures = 10000

tokenizer = Tokenizer(num_words=max_fatures, split=' ')

tokenizer.fit_on_texts(data['text'].values)

X = tokenizer.texts_to_sequences(data['text'].values)

X = pad_sequences(X)

In [10]:
embed_dim = 128
lstm_out = 98

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(7, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 33, 128)           1280000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 33, 128)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 98)                88984     
                                                                 
 dense_1 (Dense)             (None, 7)                 693       
                                                                 
Total params: 1,369,677
Trainable params: 1,369,677
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
Y = pd.get_dummies(data['label']).values

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)

print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7504, 33) (7504, 7)
(3696, 33) (3696, 7)


In [12]:
batch_size = 128
model.fit(X_train, Y_train, epochs = 10, batch_size=batch_size, verbose = 1, validation_split=0.3)

Epoch 1/10
42/42 [==============================] - 133s 3s/step - loss: 1.9269 - accuracy: 0.1929 - val_loss: 1.8997 - val_accuracy: 0.2633
Epoch 2/10
42/42 [==============================] - 132s 3s/step - loss: 1.7709 - accuracy: 0.3463 - val_loss: 1.7291 - val_accuracy: 0.3597
Epoch 3/10
42/42 [==============================] - 131s 3s/step - loss: 1.3932 - accuracy: 0.5217 - val_loss: 1.5101 - val_accuracy: 0.4267
Epoch 4/10
42/42 [==============================] - 128s 3s/step - loss: 0.9862 - accuracy: 0.6674 - val_loss: 1.3989 - val_accuracy: 0.4867
Epoch 5/10
42/42 [==============================] - 128s 3s/step - loss: 0.7044 - accuracy: 0.7761 - val_loss: 1.4385 - val_accuracy: 0.4880
Epoch 6/10
42/42 [==============================] - 125s 3s/step - loss: 0.5092 - accuracy: 0.8435 - val_loss: 1.4539 - val_accuracy: 0.4920
Epoch 7/10
42/42 [==============================] - 127s 3s/step - loss: 0.3895 - accuracy: 0.8810 - val_loss: 1.7244 - val_accuracy: 0.4734
Epoch 8/10
42

In [ ]:
score, acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)

print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

19/19 - 3s - loss: 1.4535 - accuracy: 0.5030 - 3s/epoch - 136ms/step
score: 1.45
acc: 0.50


In [ ]:
classes = { 'anger': 0,
            'fear': 1,
            'happiness': 2,
            'love': 3,
            'rq': 4,
            'sadness': 5,
            'worry': 6}

tokenizer = Tokenizer(num_words=max_fatures, split=' ')

#filename = 'sa_model_i1.sav'
#model = pickle.load(open(filename, 'rb'))

# Function to decode prediction to a class
def get_text_prediction(text):
    text = str(text)
    # vectorizing the tweet by the pre-fitted tokenizer instance
    vector = tokenizer.texts_to_sequences(text)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    vector = pad_sequences(vector, maxlen=34, dtype='int32', value=0)
    sentiment = model.predict(vector, batch_size=1, verbose = 2)[0]
    sentiment = np.argmax(sentiment)
    sentiment = [k for k, v in classes.items() if v == sentiment]
    print('Input text: ', text)
    print('Predicted class: ', sentiment)

In [ ]:
#get_text_prediction('we focus only on positive aspects and integration.')
get_text_prediction('love you!')

9/9 - 1s - 1s/epoch - 118ms/step
Input text:  love you!
Predicted class:  ['rq']


In [ ]:
# save the model to disk
#filename = 'sa_model_i1.sav'
pickle.dump(model, open(filename, 'wb'))

INFO:tensorflow:Assets written to: ram://171d4718-c6d0-4003-bf09-51ca115ba69b/assets


INFO:tensorflow:Assets written to: ram://171d4718-c6d0-4003-bf09-51ca115ba69b/assets
